### Predictive Modeling for Player Performance

#### Overview  
This notebook focuses on forecasting player performance for Partizan Mozzart Bet Belgrade’s 2022/23 roster by predicting points scored in a game using historical data. I developed regression models - Linear Regression and Random Forest - to predict points based on features like playing time, recent performance, game phase, and home/away status. Additionally, I built player-specific models for key players **Kevin Punter** and **Mathias Lessort** to capture individual trends. The goal is to identify the best predictive approach, providing actionable insights for game planning and strategy optimization.

In [3]:
import pandas as pd

file_path = "../data/partizan_2022_cleaned.csv"

df = pd.read_csv(file_path)

Filtering ensures the dataset focuses on players with consistent participation, providing a robust foundation for modeling. The 10-game threshold balances sample size and inclusion, minimizing noise from players with insufficient data.

In [4]:
# filter active players with sufficient games (e.g., at least 10 games)
players = df[df['player'] != 'PARTIZAN MOZZART BET BELGRADE'].copy()
active_players = players[players['is_playing'] == True].copy()
player_games = active_players.groupby('player')['game'].nunique()
active_players = active_players[active_players['player'].isin(player_games[player_games >= 10].index)]

Prepare the dataset by engineering features to capture recent performance, game context, and playing time, ensuring the data is suitable for predictive modeling.

In [5]:
# prepare data
# create a numerical phase order column (0 for regular season, 1 for playoffs)
active_players['phase_order'] = active_players['phase'].map({'REGULAR SEASON': 0, 'PLAYOFFS': 1})

# sort by player, phase_order, and round to ensure chronological order
active_players = active_players.sort_values(['player', 'phase_order', 'round'])

# add points_previous_game
active_players['points_previous_game'] = active_players.groupby('player')['points'].shift(1)

# drop rows with missing lagged data
active_players = active_players.dropna(subset=['points_previous_game'])

# add is_playoff column
active_players['is_playoff'] = (active_players['phase'] == 'PLAYOFFS').astype(int)

# add is_home feature (1 if Partizan is home, 0 if away)
active_players['is_home'] = active_players['game'].apply(lambda x: 1 if x.startswith('PAR') else 0)

# drop the temporary phase_order column
active_players = active_players.drop(columns=['phase_order'])

# check a few rows for a player
print(active_players[active_players['player'] == 'Punter Kevin'][['player', 'game', 'round', 'phase', 'points', 'points_previous_game']].head())

          player     game  round           phase  points  points_previous_game
23  Punter Kevin  BAS-PAR      2  REGULAR SEASON      26                  13.0
31  Punter Kevin  PAR-MIL      3  REGULAR SEASON       7                  26.0
33  Punter Kevin  PAR-VIR      4  REGULAR SEASON      10                   7.0
35  Punter Kevin  PAR-ZAL      5  REGULAR SEASON      26                  10.0
27  Punter Kevin  PAN-PAR      6  REGULAR SEASON      29                  26.0


### Interpretation

#### Feature Engineering

- **points_previous_game**: Captures recent form (e.g., Punter’s 26 points followed by 7, showing variability).
- **is_playoff**: Accounts for game intensity differences.
- **is_home**: Reflects home advantage (e.g., 'PAR-MIL' is a home game).
- **Chronological sorting**: Ensures lagged features align correctly with game sequences.
- **Dropping missing data**: Maintains data integrity, though it reduces the dataset size slightly.

The prepared dataset is now ready for modeling with features `total_seconds`, `points_previous_game`, `is_playoff`, and `is_home` to predict points.

Develop a baseline Linear Regression model to predict points, providing an interpretable starting point for performance forecasting.

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# features and target (now including is_home)
X = active_players[['total_seconds', 'points_previous_game', 'is_playoff', 'is_home']]
y = active_players['points']

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)


print("Linear Regression Results:")
print("Intercept:", lr_model.intercept_)
print("Coefficients (total_seconds, points_previous_game, is_playoff, is_home):", lr_model.coef_)

# predict and evaluate Linear Regression
y_pred_lr = lr_model.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f"R-squared: {r2_lr:.2f}")
print(f"Mean Squared Error: {mse_lr:.2f}")

Linear Regression Results:
Intercept: -1.4678340138664883
Coefficients (total_seconds, points_previous_game, is_playoff, is_home): [ 0.00854191 -0.02812799 -0.10202428  0.48724258]
R-squared: 0.36
Mean Squared Error: 29.25


### Interpretation

#### Equation
`points = -1.47 + 0.0085 * total_seconds - 0.0281 * points_previous_game - 0.102 * is_playoff + 0.487 * is_home`

#### Coefficients
- **total_seconds**: `0.0085` (0.51 points per minute) - playing time significantly impacts scoring.
- **points_previous_game**: `-0.028` - higher previous scores slightly reduce predicted points, possibly due to regression to the mean or opponent adjustments.
- **is_playoff**: `-0.102` - playoff games slightly lower points, reflecting increased competition.
- **is_home**: `0.487` - home games provide a small scoring boost, confirming a home advantage.

#### Performance
- **R² = 0.36**: The model explains 36% of the variance in points, a moderate fit.
- **MSE = 29.25**: Average prediction error of ~5.41 points (RMSE), reasonable for a baseline.

The model captures key trends (e.g., playing time’s importance) but leaves 64% of variance unexplained, suggesting potential for more complex models or additional features.

Explore a Random Forest model to capture potential non-linear relationships and feature interactions, aiming to improve prediction accuracy over Linear Regression.

In [7]:
from sklearn.ensemble import RandomForestRegressor

# train the Random Forest model
rf_model = RandomForestRegressor(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)

# predict and evaluate Random Forest
y_pred_rf = rf_model.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("\nRandom Forest Results:")
print(f"R-squared: {r2_rf:.2f}")
print(f"Mean Squared Error: {mse_rf:.2f}")
print("Feature Importance (total_seconds, points_previous_game, is_playoff, is_home):", rf_model.feature_importances_)


Random Forest Results:
R-squared: 0.29
Mean Squared Error: 32.50
Feature Importance (total_seconds, points_previous_game, is_playoff, is_home): [0.8080132  0.15034656 0.01366977 0.02797047]


### Interpretation

#### Performance
- **R² = 0.29**: Explains 29% of variance, worse than Linear Regression (0.36).
- **MSE = 32.50**: Average error of ~5.70 points (RMSE), higher than Linear Regression (5.41).

#### Feature Importance
- **total_seconds**: 80.8% - dominates predictions, aligning with Linear Regression’s findings.
- **points_previous_game**: 15.0% - notable but secondary.
- **is_playoff**: 1.4% - minimal impact.
- **is_home**: 2.8% - minimal impact.

The Random Forest underperforms, likely due to overfitting on a small dataset (after filtering) or because the relationships are mostly linear, favoring Linear Regression’s simpler assumptions.

## Results

| Model             | R-squared | Mean Squared Error | RMSE (Points) |
|-------------------|-----------|--------------------|---------------|
| Linear Regression | 0.36      | 29.25              | 5.41          |
| Random Forest     | 0.29      | 32.50              | 5.70          |

## Interpretation
- **Best Model**: Linear Regression outperforms Random Forest (R²=0.36 vs. 0.29, RMSE=5.41 vs. 5.70).
- **Reason**: The small dataset and predominantly linear relationships favor Linear Regression’s simplicity. Random Forest may have overfit due to limited data, as it’s more suited to larger datasets with complex patterns.
- **Practical Use**: Linear Regression provides a reasonable prediction with an average error of 5.41 points, useful for rough forecasts (e.g., “Punter might score 15 points if he plays 30 minutes”).

Develop tailored Linear Regression models for Kevin Punter and Mathias Lessort to capture individual scoring patterns, potentially improving prediction accuracy over the general model.

In [8]:
# function to train and evaluate a player-specific model
def train_player_model(player_name):
    # filter data for the player
    player_data = active_players[active_players['player'] == player_name]
    
    # check if the player has enough data
    if len(player_data) < 10:
        print(f"Not enough data for {player_name} (only {len(player_data)} games).")
        return
    
    # features and target
    X_player = player_data[['total_seconds', 'points_previous_game', 'is_playoff', 'is_home']]
    y_player = player_data['points']
    
    # split into train and test sets
    X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_player, y_player, test_size=0.2, random_state=42)
    
    # train the model
    model_player = LinearRegression()
    model_player.fit(X_train_p, y_train_p)
    
    # predict and evaluate
    y_pred_p = model_player.predict(X_test_p)
    r2_p = r2_score(y_test_p, y_pred_p)
    mse_p = mean_squared_error(y_test_p, y_pred_p)
    
    
    
    print(f"\nResults for {player_name}:")
    print("Intercept:", model_player.intercept_)
    print("Coefficients (total_seconds, points_previous_game, is_playoff, is_home):", model_player.coef_)
    print(f"R-squared: {r2_p:.2f}")
    print(f"Mean Squared Error: {mse_p:.2f}")

# train models for Kevin Punter and Mathias Lessort
train_player_model('Punter Kevin')
train_player_model('Lessort Mathias')


Results for Punter Kevin:
Intercept: 0.27180171401723996
Coefficients (total_seconds, points_previous_game, is_playoff, is_home): [ 0.01135243 -0.24435824  4.5532027   0.90994255]
R-squared: 0.61
Mean Squared Error: 12.88

Results for Lessort Mathias:
Intercept: 5.645722623747675
Coefficients (total_seconds, points_previous_game, is_playoff, is_home): [ 0.00617132 -0.43831527 -0.15918573  2.36874769]
R-squared: -0.27
Mean Squared Error: 17.97


### Interpretation

#### Kevin Punter
- **Role**: Scoring guard, known for offensive consistency.
- **Equation**: `points = 0.272 + 0.0114 * total_seconds - 0.244 * points_previous_game + 4.553 * is_playoff + 0.910 * is_home`
- **Coefficients**:
  - **total_seconds**: `0.0114` (0.68 points per minute) - strong reliance on playing time.
  - **points_previous_game**: `-0.244` - suggests variability after high-scoring games, possibly due to opponent adjustments.
  - **is_playoff**: `4.553` - significant boost in playoffs, reflecting clutch performance.
  - **is_home**: `0.910` - notable home advantage, higher than the general model’s 0.487.
- **Performance**: R²=0.61 (61% variance explained), MSE=12.88 (RMSE=3.59 points).
- **Insight**: The model fits well, significantly improving over the general model (R²=0.36, RMSE=5.41), making it reliable for predicting Punter’s scoring with an error of ~3.6 points.

#### Mathias Lessort
- **Role**: Dominant big man, likely matchup-dependent.
- **Equation**: `points = 5.646 + 0.0062 * total_seconds - 0.438 * points_previous_game - 0.159 * is_playoff + 2.369 * is_home`
- **Coefficients**:
  - **total_seconds**: `0.0062` (0.37 points per minute) - less dependent on time than Punter.
  - **points_previous_game**: `-0.438` - high variability, possibly due to inconsistent minutes or matchups.
  - **is_playoff**: `-0.159` - slight dip in playoffs.
  - **is_home**: `2.369` - large home boost, much higher than Punter’s 0.910.
- **Performance**: R²=-0.27 (worse than a mean prediction), MSE=17.97 (RMSE=4.24 points).
- **Insight**: The model performs poorly, failing to capture Lessort’s scoring patterns, though its RMSE (4.24) is better than the general model’s 5.41. Lessort’s scoring may be too matchup-dependent for these features.

### Comparison
- Punter’s model significantly outperforms the general model, showing the value of player-specific modeling for key scorers.
- Lessort’s model fails, indicating his scoring is harder to predict with these features, possibly due to external factors like opponent defense.

## Conclusion

### Key Findings

#### General Models
- Linear Regression (R²=0.36, RMSE=5.41) outperforms Random Forest (R²=0.29, RMSE=5.70), providing a solid baseline for predicting points across all players.
- Playing time (`total_seconds`) is the most significant predictor in both models.

#### Player-Specific Models
- **Kevin Punter**: The tailored model excels (R²=0.61, RMSE=3.59), capturing his scoring trends effectively and improving over the general model.
- **Mathias Lessort**: The model performs poorly (R²=-0.27, RMSE=4.24), suggesting his scoring is influenced by factors beyond the current features.

#### Feature Insights
- Home advantage and playoff intensity impact scoring, with varying effects across players.
- Recent performance (`points_previous_game`) often shows a negative coefficient, indicating variability or regression to the mean.

#### Strategic Implications
- **Game Planning**: Use Punter’s model for reliable predictions (e.g., adjust minutes to target specific scoring outputs).
- **Team Strategy**: Leverage home advantage (especially for Lessort) and prepare for playoff intensity (notably for Punter’s scoring boost).